In [1]:
from tensorflow.keras.models import load_model
import joblib
import numpy as np

# Load MLP Model
mlp = load_model("models/mlp_model.h5")

# Load XGBoost Model
xgb_model = joblib.load("models/xgb_model.pkl")

# Load Scaler
scaler = joblib.load("models/scaler.pkl")


In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
# Define attack categories (0 = Normal, 1 = DoS, 2 = Probe, 3 = R2L)
#labels = [0, 1, 2, 3]

# Create inference data with 10 samples (5 normal, 5 attacks)
inference_data = pd.DataFrame({
    "duration": np.random.randint(0, 500, 10),
    "protocol_type": np.random.choice([0, 1, 2], 10),  # Encoded categories
    "service": np.random.randint(0, 66, 10),  # Encoded service types
    "flag": np.random.randint(0, 11, 10),  # Encoded flag types
    "src_bytes": np.random.randint(0, 10000, 10),
    "dst_bytes": np.random.randint(0, 10000, 10),
    "land": np.random.choice([0, 1], 10),
    "wrong_fragment": np.random.randint(0, 3, 10),
    "urgent": np.random.randint(0, 3, 10),
    "hot": np.random.randint(0, 5, 10),
    "num_failed_logins": np.random.randint(0, 5, 10),
    "logged_in": np.random.choice([0, 1], 10),
    "lnum_compromised": np.random.randint(0, 10, 10),
    "lroot_shell": np.random.choice([0, 1], 10),
    "lsu_attempted": np.random.choice([0, 1, 2], 10),
    "lnum_root": np.random.randint(0, 10, 10),
    "lnum_file_creations": np.random.randint(0, 5, 10),
    "lnum_shells": np.random.randint(0, 5, 10),
    "lnum_access_files": np.random.randint(0, 5, 10),
    "lnum_outbound_cmds": np.random.randint(0, 5, 10),
    "is_host_login": np.random.choice([0, 1], 10),
    "is_guest_login": np.random.choice([0, 1], 10),
    "count": np.random.randint(0, 500, 10),
    "srv_count": np.random.randint(0, 500, 10),
    "serror_rate": np.random.rand(10),
    "srv_serror_rate": np.random.rand(10),
    "rerror_rate": np.random.rand(10),
    "srv_rerror_rate": np.random.rand(10),
    "same_srv_rate": np.random.rand(10),
    "diff_srv_rate": np.random.rand(10),
    "srv_diff_host_rate": np.random.rand(10),
    "dst_host_count": np.random.randint(0, 500, 10),
    "dst_host_srv_count": np.random.randint(0, 500, 10),
    "dst_host_same_srv_rate": np.random.rand(10),
    "dst_host_diff_srv_rate": np.random.rand(10),
    "dst_host_same_src_port_rate": np.random.rand(10),
    "dst_host_srv_diff_host_rate": np.random.rand(10),
    "dst_host_serror_rate": np.random.rand(10),
    "dst_host_srv_serror_rate": np.random.rand(10),
    "dst_host_rerror_rate": np.random.rand(10),
    "dst_host_srv_rerror_rate": np.random.rand(10),
    #"label": np.random.choice(labels, 10, p=[0.5, 0.5])  # 50% Normal, 50% Attack
})

    #"label": np.random.choice(labels, 10, p=[0.5, 0.3, 0.1, 0.1])  # Weighted distribution


# Save for future testing
inference_data.to_csv("inference_data.csv", index=False)

print("Inference DataFrame created and saved as 'inference_data.csv'.")


Inference DataFrame created and saved as 'inference_data.csv'.


In [3]:
inference_data.head(4)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,332,2,2,0,3867,6026,0,0,0,3,...,177,450,0.891114,0.195861,0.136970,0.565727,0.841134,0.626667,0.502503,0.370755
1,324,2,55,7,6806,4422,0,1,2,4,...,268,245,0.655648,0.799464,0.426250,0.312144,0.341289,0.045993,0.033734,0.958525
2,369,2,20,0,4819,599,0,2,1,0,...,228,10,0.492601,0.066597,0.831152,0.769862,0.415995,0.071601,0.882989,0.195419
3,85,1,39,10,8745,1002,0,0,1,3,...,390,155,0.522225,0.918709,0.170784,0.304267,0.173774,0.300392,0.415648,0.848952


In [4]:
# Standardize using the saved scaler
inference_data = scaler.transform(inference_data)


In [5]:
def logistic_map(size, r=3.99, seed=0.5):
    chaos = np.zeros(size)
    chaos[0] = seed
    for i in range(1, size):
        chaos[i] = r * chaos[i-1] * (1 - chaos[i-1])
    return chaos

# Apply the chaotic sequence to inference data
chaos_sequence = logistic_map(len(inference_data))
inference_data = np.multiply(inference_data, chaos_sequence[:, np.newaxis])


In [6]:
# MLP model prediction
mlp_pred = (mlp.predict(inference_data) > 0.5).astype(int).flatten()

1/1 [==============================] - 0s 144ms/step


In [7]:
# XGBoost model prediction
xgb_pred = xgb_model.predict(inference_data)


In [8]:
# Hybrid Fusion (Weighted Average)
final_pred = (0.6 * mlp_pred + 0.4 * xgb_pred) > 0.5

In [9]:
#Creating a mapping from 0 to 'low' and 1 to 'high'
mapping = {0: '0-Normal', 1:'1-Attack'}

#print the predictions with labels
for i, final_pred in enumerate(final_pred):
    label = mapping[final_pred]
    print(f'Data{i+1}: Predicted Outcome- {label}')


# Display predictions
print("Predictions:", final_pred)

Data1: Predicted Outcome- 0-Normal
Data2: Predicted Outcome- 0-Normal
Data3: Predicted Outcome- 0-Normal
Data4: Predicted Outcome- 0-Normal
Data5: Predicted Outcome- 0-Normal
Data6: Predicted Outcome- 0-Normal
Data7: Predicted Outcome- 0-Normal
Data8: Predicted Outcome- 0-Normal
Data9: Predicted Outcome- 0-Normal
Data10: Predicted Outcome- 0-Normal
Predictions: False
